## Focalloss 代碼解析


### CrossEntrypy

- $CE(P_t) = -log(pt)$

$pt = \frac{e^{x[class]}}{\sum_j e^{x[j]}}$ 　　經過softmax運算



$Loss(x, class) = -log(\frac{e^{x[class]}}{\sum_j e^{x[j]}}\quad)^{\, \, \gamma} $


### Focal Loss


- $FL(P_t) = -\alpha(1-p_t)^rlog(pt)$

$Loss(x, class) = -\alpha_class ( 1- \frac{e^{x[class]}}{\sum_j e^{x[j]}}\,\,\,\,    )^{\gamma}log(\frac{e^{x[class]}}{\sum_j e^{x[j]}}\,\,\,\,)$

　　　　　　　= $-\alpha_{class} ( 1- softmax(x)[class])^\gamma * log(softmax(x)[class])$
       

       
$
\alpha_t =\begin{cases}
\alpha ,  \quad if\quad  y =１\\
1 - \alpha, \quad otherwise
\end{cases}
$

- $\gamma$ 负责降低简单样本的损失值, 以解决加总后负样本loss
- $\alpha$ 调和正负样本的不平均，如果设置0.25, 那么就表示负样本为0.75, 对应公式$1 - \alpha$

### 从公式可以看出

控制样本权重的为 $\alpha(1-p_t)^\gamma$

当$p_t$越大，赋予的权重就越小， $p_t$越小，赋予的权重就越大




---
### 解决问题

基于原来多分类損失函數CrossEntropy进行改进，最初one-stage目标检测框架有easy-example（背景） 和 hard-example（前景）严重样本分布不均的问题，往往easy-example的loss与hard-example的存在极大的不平衡(1：1000)，导致模型都在学习easy-example而忽略了hard-example

<img src="https://github.com/Stephenfang51/Focal_loss_turtorial/blob/master/images/focal_gamma.png?raw=true" width=800>



根据图表，基于CE的公式， 提出了新的因子 $-(1-p_t)^\gamma$,  当 $\gamma$ 值>0, 减低了easy_example(pt>0.5)的loss值，因此模型能够更专注在学习hard_example

PS.  作者发现$\gamma$ 的初始值2为最佳


### 举个简单的例子帮助我们快速理解



假设我们模型分类负样本10000笔资料，probability(pt) = 0.95， 这边可以理解为easy-example因为概率高

正样本10笔资料， probability(pt) = 0.05， 可以理解为hard-example 概率低

直接带入CE和FL

1. 带入CrossEntropy - $CE(P_t) = -log(pt)$

    - 负样本 ： log(p_t) * 样本数（100000） = 0.02227 * 100000 = 2227
    - 正样本 ： log(p_t) * 样本数（10） = 1.30102 * 10 = 13.0102
    total loss = 2227+13.0102 = 2240
    正样本占比：13.0102 / 2240 = 0.0058
    
<br>
<br>
<br>
    
2. 带入Focalloss

假设alpha = 0.25（正样本， gamma=2


    - 负样本 ： 0.75*（1-0.95)^2 * 0.02227 *样本数（100000） = 0.00004176 * 100000 = 4.1756
    - 正样本 ： 0.25* (1-0.05)^2 * 1.30102 *样本数（10）= 0.29354264 * 10 = 2.935
    total loss = 4.175 + 2.935 = 7.110
    正样本占比：2.935/7.110 = 0.4127（与0.0058差距甚大）



### 小结：

1. gamma = 2时候， 负样本 $(1-0.95)^2$  = 0.0025， 正样本$(1-0.05)^2$ = 0.9025, 负样本损失值明显比正样本小很多
2. alpha与gamma是一种相互平衡的值，虽然就理论上来看，alpha值设定为0.75(因为正样本通常数量小)是比较合理， 但是配合gamma值已经将负样本损失值降低许多，可理解为alpha和gamma相互牵制，alpha也不让正样本占比太大，因此最终设定为0.25

    
<br>
<br>
<br>
<br>
    
    
    
    
论文连接 https://arxiv.org/abs/1708.02002

pytorch源碼实践 https://github.com/marvis/pytorch-yolo2/blob/master/FocalLoss.py

In [229]:
import torch as t
import torch.nn.functional as F
import torch.nn as nn

$L = -\sum^C_i onehot \bigotimes \alpha(1 - P_i)^\gamma log P_i$

$\bigotimes$ = element-wise 乘法运算


整个代码执行思路围绕上述公式，简单简洁

### 代码执行思路

   1. 首先取得$(P_i)$：对input进行softmax，取得概率(probability)
   2. 基于input的shape制作class_mask(one_hot) 元素全为0
   3. 将targets(label)的索引值丢回class_mask，就生成了one_hot形式的targets（目标值为1, 其余全为0）ex.[0, 0, 0, 1, 0]
   4. $[(P_i) \bigotimes classmask(onehot)]$ 并且求log对数 = $log_{pi}$ $\bigotimes$ 表示element-wise
   5. 带回公式， 此时$log_{P_i}$ 已经得到， alpha值和gamma值作为超参数传入
       - $L = -\sum^C_i onehot \bigotimes \alpha(1 - P_i)^\gamma log P_i$

In [230]:
class FocalLoss(nn.Module):
    def __init__(self, class_num, alpha=None, gamma=2, size_average=True):
        super(FocalLoss, self).__init__()
        if alpha is None:
            self.alpha = t.ones(class_num, 1)
        else:
            if alpha:
#                 self.alpha = t.ones(class_num, 1, requires_grad=True)
                self.alpha = t.tensor(alpha, requires_grad=True)
                print('alpha初始\n', self.alpha)
                print('alpha shape\n', self.alpha.shape)
#             else:
#                 self.alpha = t.ones(class_num, 1*alpha).cuda()
        self.gamma = gamma
        self.class_num = class_num
        self.size_average = size_average
        
    def forward(self, inputs, targets):
        #input.shape = (N, C)
        N = inputs.size(0)
        C = inputs.size(1)
        P = F.softmax(inputs) #經過softmax 概率
        #---------one hot start--------------#
        class_mask = inputs.data.new(N, C).fill_(0)  #生成和input一样shape的tensor
        print('依照input shape制作:class_mask\n', class_mask)
        class_mask = class_mask.requires_grad_() #需要更新， 所以加入梯度计算
        ids = targets.view(-1, 1) #取得目标的索引
        print('取得targets的索引\n', ids)
        class_mask.data.scatter_(1, ids.data, 1.) #利用scatter将索引丢给mask
        print('targets的one_hot形式\n', class_mask) #one-hot target生成
        #---------one hot end-------------------#
        if inputs.is_cuda and not self.alpha.is_cuda:
            self.alpha = self.alpha.cuda()
#         alpha = self.alpha[ids.data.view(-1, 1)]
#         alpha = self.alpha[ids.view(-1)]
        alpha = self.alpha
        print('alpha值\n', alpha)
        print('alpha shape\n', alpha.shape)
        
        probs = (P*class_mask).sum(1).view(-1, 1) 
        print('留下targets的概率（1的部分），0的部分消除\n', probs)
        #将softmax * one_hot 格式，0的部分被消除 留下1的概率， shape = (5, 1), 5就是每个target的概率
        
        log_p = probs.log()
        print('取得对数\n', log_p)
        #取得对数
        
        batch_loss = -alpha*(t.pow((1-probs), self.gamma))*log_p #對應下面公式
        print('每一个batch的loss\n', batch_loss)
        #batch_loss就是取每一个batch的loss值
        
        
        #最终将每一个batch的loss加总后平均
        if self.size_average:
            loss = batch_loss.mean()
        else:
            loss = batch_loss.sum()
        print('loss值为\n', loss)
        return loss

# 带入模拟数值看结果 - Focalloss

In [237]:
t.manual_seed(50) #随机种子确保每次input tensor值是一样的
input = torch.randn(5, 5, dtype=torch.float32, requires_grad=True)
print('input值为\n', input)
targets = t.randint(5, (5, ))
print('targets值为\n', targets)

input值为
 tensor([[-1.1588,  0.3673,  0.7110, -0.2373, -1.0129],
        [ 0.5580, -0.8784, -1.1446, -0.7629, -0.0170],
        [-0.0477,  0.1770, -0.6058,  0.0125,  0.6818],
        [ 0.4508,  0.4299,  1.0491,  0.0453, -1.5092],
        [-1.5502, -0.7564,  0.6274, -0.4808, -0.6856]], requires_grad=True)
targets值为
 tensor([1, 0, 4, 3, 4])


In [243]:
criterion = FocalLoss(5, alpha=0.25, gamma=2, size_average=True)
loss = criterion(input, targets)
loss.backward()
# print(input.grad.data)

alpha初始
 tensor(0.2500, requires_grad=True)
alpha shape
 torch.Size([])
依照input shape制作:class_mask
 tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])
取得targets的索引
 tensor([[1],
        [0],
        [4],
        [3],
        [4]])
targets的one_hot形式
 tensor([[0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.]], requires_grad=True)
alpha值
 tensor(0.2500, requires_grad=True)
alpha shape
 torch.Size([])
留下targets的概率（1的部分），0的部分消除
 tensor([[0.2920],
        [0.4445],
        [0.3480],
        [0.1447],
        [0.1370]], grad_fn=<ViewBackward>)
取得对数
 tensor([[-1.2312],
        [-0.8108],
        [-1.0556],
        [-1.9329],
        [-1.9875]], grad_fn=<LogBackward>)
每一个batch的loss
 tensor([[0.1543],
        [0.0625],
        [0.1122],
        [0.3535],
        [0.3700]], grad_fn=<MulBackward0>)
loss值为
 ten

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


# 带入Cross_entropy

In [219]:
F.cross_entropy(input, targets)

tensor(1.4036)

###  以上能明显感觉到一开始的loss值已经有很大差距